### 1. 함수형태로 제작
### 2. 진행과정을 눈으로 확인해보기
### 1. 필요 모듈 불러오기

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision.datasets import CIFAR10

### 2. 데이터 불러오기

In [2]:
# 데이터셋 로드 (정규화 없이)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG는 224x224 크기의 입력
    transforms.ToTensor()
])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)


Files already downloaded and verified


- 2-1 평균과 표준편차 구하기

In [3]:
# 평균과 표준편차 계산
def get_mean_std(loader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0

    for data, _ in loader:
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(data**2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = (channels_squared_sum / num_batches - mean**2)**0.5
    return mean, std

mean, std = get_mean_std(trainloader)
print(f"Mean: {mean}")
print(f"Std: {std}")

Mean: tensor([0.4915, 0.4822, 0.4466])
Std: tensor([0.2413, 0.2378, 0.2565])


In [4]:
# 데이터셋 로드 (정규화)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG는 224x224 크기의 입력
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4821, 0.4465), std=(0.2413, 0.2378, 0.2564))
])

In [5]:
dataset = CIFAR10(root='../data', train=True, download=True, transform=transform)
testset = CIFAR10(root="../data", train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
len(dataset), len(testset)

(50000, 10000)

In [7]:
trainset, valset = random_split(dataset, [int(len(dataset)*0.8), int(len(dataset)*0.2)])

In [8]:
len(trainset), len(valset), len(testset)

(40000, 10000, 10000)

In [9]:
batch_size = 64

# loader 만들기
train_loader = DataLoader(trainset, batch_size = batch_size, shuffle=True) 
val_loader = DataLoader(valset, batch_size = batch_size, shuffle=False) 
test_loader = DataLoader(testset, batch_size = batch_size, shuffle=False) 

### 3. 모델 설정하기

In [10]:
model_ResNet = models.resnet18(weights=None, num_classes=10)

### 4. 손실함수와 옵티마이저 설정

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_ResNet.parameters(), lr=0.01)

In [12]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### 5.학습함수 정의

In [14]:
import torchmetrics
from tqdm.notebook import tqdm

# train 과 val 과정중의 loss값과 accuracy 값을 저장하는것이 목표
def train_and_validate(model, train_loader, val_loader, criterion, epochs, device):
    train_losses = []
    val_losses = []

    train_accuracies = []
    val_accuracies = []

    model = model.to(device)
    
    # 옵티마이저 다시 설정
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(epochs):
        # 모델 학습용으로 설정
        model.train()
        running_loss = 0.0

        train_accuracy_metrics = torchmetrics.Accuracy(task="multiclass", num_classes=10, average='macro').to(device)

        # 학습 진행해보기
        for inputs, labels in tqdm(train_loader, desc=f"{epoch} 별 학습 진행중"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 그라디언트 초기화
            optimizer.zero_grad()
            outputs = model(inputs) # 결과
            loss = criterion(outputs, labels)
            loss.backward() # 역전파
            optimizer.step()

            running_loss += loss.item()
            
            # 학습 정확도 계산
            train_accuracy_metrics.update(outputs, labels)
        
        # 에포크별 학습손실 계산
        epoch_loss = running_loss / len(train_loader)
        train_losses.append(epoch_loss)

        # 정확도 계산
        train_accuracy = train_accuracy_metrics.compute().item()
        train_accuracies.append(train_accuracy)

        print(f"에포크별 학습 loss : {epoch_loss:.3f} / 학습 정확도 : {train_accuracy:.2%}")

    return train_losses, val_losses, train_accuracies, val_accuracies

### 7. 모델학습

In [15]:
device = torch.device("cuda")
history = train_and_validate(model_ResNet, train_loader, val_loader, criterion, epochs=10, device=device)

0 별 학습 진행중:   0%|          | 0/625 [00:00<?, ?it/s]

에포크별 학습 loss : 1.774 / 학습 정확도 : 34.48%


1 별 학습 진행중:   0%|          | 0/625 [00:00<?, ?it/s]

에포크별 학습 loss : 1.278 / 학습 정확도 : 53.59%


2 별 학습 진행중:   0%|          | 0/625 [00:00<?, ?it/s]

...

에포크별 학습 loss : 0.183 / 학습 정확도 : 93.55%


8 별 학습 진행중:   0%|          | 0/625 [00:00<?, ?it/s]

에포크별 학습 loss : 0.124 / 학습 정확도 : 95.65%


9 별 학습 진행중:   0%|          | 0/625 [00:00<?, ?it/s]

에포크별 학습 loss : 0.104 / 학습 정확도 : 96.30%
